In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

import sys
sys.path.append('..')
import src

In [2]:
PARAMS = {
    'community': 'laurelhurst',
    'experiment_name': 'resource_level',
    # resource types should be the same as the column names in the resource_preference_pmf and resource_inventory_pmf
    'resource_types': ['water', 'food', 'meds', 'transp', 'comm',
                       'faid', 'warm', 'sani', 'power', 'shelter'], 
    'tau': 5, # isolation days
    'num_scenarios': 100,
    'n': 0.78773808, # degree distribution parameter
    'p': 0.02559534, # degree distribution parameter
    'social_tie': {
        'types': ['strong', 'weak'],
        'pmf':  [0.41296, 0.58704]
    },
    'distance_decay_alpha': -1.35, # distance decay parameter
    'distances_mat': np.load('../temp/laurelhurst_distance_matrix.npy'), # distance matrix
    'sharing_preference_pmf': pd.read_csv('../temp/laurelhurst_share_pref_PMF.csv'), # sharing preference matrix
    'sharing_priority': [3, 2, 1], # sharing priorities to [strong tie, weak tie, stranger]
    'resource_inventory_pmf': pd.read_csv('../temp/laurelhurst_resource_distr_PMF.csv') # resource inventory distribution
}

In [3]:
def p2p_single_resource_sharing(resource:str, 
                                tau:float, # isolation days
                                n:float, # negative binomial distribution parameter
                                p:float, # negative binomial distribution parameter
                                resource_level:float, # resource level
                                distance_decay_alpha:float, # distance decay parameter
                                distance_mat:np.matrix, # distance matrix
                                sharing_preference_PMF:pd.Series, 
                                resource_inventory_PMF:pd.Series,
                                social_tie_pmf:list,
                                social_tie_types:list = ['strong', 'weak'],
                                sharing_priority:list = [3, 2, 1],
                                seed:int=0):
    """Step 1: Community-based social network construction"""
    # create a community
    community = src.Community()
    num_nodes = distance_mat.shape[0]
    community.add_nodes_from(range(num_nodes))

    # generate social ties
    np.random.seed(seed)
    degree_list = np.random.negative_binomial(n, p, size=num_nodes)
    community.generate_social_ties(degrees=degree_list, 
                                   distance_matrix=distance_mat, 
                                   distance_decay_alpha=distance_decay_alpha, 
                                   seed=seed+1)
    community.split_social_ties(types=social_tie_types, 
                                pmf=social_tie_pmf, 
                                seed=seed+2)

    """Step 2: Community-based resource-sharing network construction"""
    # generate sharing preferences
    community.generate_sharing_preference(recipient=[0, 1, 2, 3], # [nobody, strong tie only, strong and weak ties only, everybody]
                                          pmf=sharing_preference_PMF, 
                                          name=resource+'_share_preference',
                                          seed=seed+3)

    """Step 3: Community-based resource inventory"""
    community.generate_resource_inventory(support=np.arange(0, len(resource_inventory_PMF)), 
                                          pmf=resource_inventory_PMF, 
                                          name=resource, 
                                          seed=seed+4)
    for n in community.nodes():
        community.nodes[n][resource] = community.nodes[n][resource] * resource_level # set resource level

    # create sharing network
    share_network = community.get_share_network(sharing_preference=resource+'_share_preference', 
                                                priority=sharing_priority)

    """Step 4: Community-based P2P resource-sharing model"""
    priority_matrix = community.get_share_priority_matrix(sharing_preference=resource+'_share_preference', 
                                                          priority=sharing_priority)
    resource_inventory = [community.nodes[n][resource] for n in community.nodes]
    model = src.ResourceSharingModel(wts_matrix=priority_matrix, 
                                     inventory=resource_inventory, 
                                     isolation_days=tau)
    model.solve()

    return community, share_network, model

In [4]:
# main script for running the status quo model
# for resource_level in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
for resource_level in [1.6]:

    for sce in range(PARAMS['num_scenarios']):
        
        # directory name for saving the results
        dir = PARAMS['community'] + '/' + PARAMS['experiment_name'] + '/tau_' + str(PARAMS['tau']) # directory name
        # Create a dataframe to store the results
        inventory_df = pd.DataFrame(columns=PARAMS['resource_types'])
        resource_df = pd.DataFrame(columns=PARAMS['resource_types'])
        
        # run the model for each resource type
        for rsc in PARAMS['resource_types']:
            community, share_network, model = p2p_single_resource_sharing(resource=rsc,
                                                                        tau=PARAMS['tau'],
                                                                        n=PARAMS['n'],
                                                                        p=PARAMS['p'],
                                                                        resource_level=resource_level,
                                                                        social_tie_types = PARAMS['social_tie']['types'],
                                                                        social_tie_pmf = PARAMS['social_tie']['pmf'],
                                                                        distance_decay_alpha = PARAMS['distance_decay_alpha'],
                                                                        distance_mat = PARAMS['distances_mat'],
                                                                        sharing_preference_PMF = PARAMS['sharing_preference_pmf'][rsc],
                                                                        sharing_priority = PARAMS['sharing_priority'],
                                                                        resource_inventory_PMF = PARAMS['resource_inventory_pmf'][rsc],
                                                                        seed=sce)
            inventory_df[rsc] = model.inventory.flatten()
            resource_df[rsc] = model.resource.flatten()

        # save the resource inventory vector in csv file
        inventory_df.to_csv(dir + '/q_vec_rsc_{}_sce_{}.csv'.format(resource_level, sce), index=False)
        # save the final resource vector in csv file
        resource_df.to_csv(dir + '/h_vec_rsc_{}_sce_{}.csv'.format(resource_level, sce), index=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-31
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10700 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3746 rows, 3508129 columns and 7016258 nonzeros
Model fingerprint: 0xc2f5f633
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 6e+00]
Presolve removed 2103 rows and 3299008 columns
Presolve time: 2.89s
Presolved: 1643 rows, 209121 columns, 418127 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.090e+05
 Factor NZ  : 4.164e+05 (roughly 90 MB of memory)
 Factor Ops : 1.492e+08 (less than 1 second per iteration)
 Threads    : 6

                